In [2]:
import UnicornPy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
import socket
from joblib import load
import pyriemann

mne.set_log_level('ERROR')

In [23]:
def preprocess_raw(stacked_epo_data,epo_length):
    fs = 250
    ch_names = ['C3','Cz','C4']
    stacked_epo_data= stacked_epo_data[1:4,:]
    info = mne.create_info(ch_names, fs)
    stacked_epo_mne = mne.io.RawArray(stacked_epo_data,info)
    stacked_epo_mne.filter(8,14,picks=ch_names)
    stacked_epo_mne.notch_filter(50,picks=ch_names)
    epo_data = stacked_epo_mne.get_data()[:,:-epo_length]
    return np.cov(epo_data)

In [24]:
TestsignaleEnabled = False;
FrameLength = 1;
AcquisitionDurationInSeconds = 1;
epoch_time = 2;
# clf = load('L_Feet_clf.joblib')
clf = load('rest_left_right_feet_classi.joblib')

In [13]:
deviceList = UnicornPy.GetAvailableDevices(True)
device = UnicornPy.Unicorn(deviceList[0])
numberOfAcquiredChannels= device.GetNumberOfAcquiredChannels()
configuration = device.GetConfiguration()

print("Acquisition Configuration:");
print("Sampling Rate: %i Hz" %UnicornPy.SamplingRate);
print("Frame Length: %i" %FrameLength);
print("Number Of Acquired Channels: %i" %numberOfAcquiredChannels);
print("Data Acquisition Length: %i s" %AcquisitionDurationInSeconds);
print();


Acquisition Configuration:
Sampling Rate: 250 Hz
Frame Length: 1
Number Of Acquired Channels: 17
Data Acquisition Length: 1 s



In [6]:
receiveBufferBufferLength = FrameLength * numberOfAcquiredChannels * 4
receiveBuffer = bytearray(receiveBufferBufferLength)
device.StartAcquisition(TestsignaleEnabled)

In [7]:
server = ('localhost', 4000)
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(server)

In [26]:
try:
    # Get available devices.
    #-------------------------------------------------------------------------------------

    # Get available device serials.
    deviceList = UnicornPy.GetAvailableDevices(True)

    if len(deviceList) <= 0 or deviceList is None:
        raise Exception("No device available.Please pair with a Unicorn first.")
    deviceID = 0
    # Open selected device.
    #-------------------------------------------------------------------------------------
    print()
    print("Trying to connect to '%s'." %deviceList[deviceID])
    device = UnicornPy.Unicorn(deviceList[deviceID])
    print("Connected to '%s'." %deviceList[deviceID])
    print()


    # Initialize acquisition members.
    #-------------------------------------------------------------------------------------
    numberOfAcquiredChannels= device.GetNumberOfAcquiredChannels()
    configuration = device.GetConfiguration()

    # Print acquisition configuration
    print("Acquisition Configuration:");
    print("Sampling Rate: %i Hz" %UnicornPy.SamplingRate);
    print("Frame Length: %i" %FrameLength);
    print("Number Of Acquired Channels: %i" %numberOfAcquiredChannels);
    print("Data Acquisition Length: %i s" %AcquisitionDurationInSeconds);
    print();

    # Allocate memory for the acquisition buffer.
    receiveBufferBufferLength = FrameLength * numberOfAcquiredChannels * 4
    receiveBuffer = bytearray(receiveBufferBufferLength)

    try:
        # Start data acquisition.
        #-------------------------------------------------------------------------------------
        device.StartAcquisition(TestsignaleEnabled)
        print("Data acquisition started.")

        # Calculate number of get data calls.
        numberOfGetDataCalls = int(AcquisitionDurationInSeconds * UnicornPy.SamplingRate / FrameLength);
    
        # Limit console update rate to max. 25Hz or slower to prevent acquisition timing issues.                   
        consoleUpdateRate = int((UnicornPy.SamplingRate / FrameLength) / 25.0);
        if consoleUpdateRate == 0:
            consoleUpdateRate = 1

        # Keeping track of last few epochs
        
        epoch_samples = epoch_time*UnicornPy.SamplingRate
        back_burner_size = 10 # epochs
        back_burner_samples = back_burner_size*epoch_samples

        back_burner = np.zeros((back_burner_samples,numberOfAcquiredChannels))
        #front_burner = np.zeros((back_burner_samples, numberOfAcquiredChannels))

        back_burner_tracker = 0

        curr_epoch = np.zeros((epoch_samples, numberOfAcquiredChannels))
        curr_epoch_tracker = 0
        # Acquisition loop.
        #-------------------------------------------------------------------------------------
        while True:
            # Receives the configured number of samples from the Unicorn device and writes it to the acquisition buffer.
            device.GetData(FrameLength,receiveBuffer,receiveBufferBufferLength)

            # Convert receive buffer to numpy float array
            data = np.frombuffer(receiveBuffer, dtype=np.float32, count=numberOfAcquiredChannels * FrameLength)
            data = np.reshape(data, (FrameLength, numberOfAcquiredChannels))

            curr_epoch[curr_epoch_tracker,:] = data
            curr_epoch_tracker += 1

            if curr_epoch_tracker == epoch_samples:
                if back_burner_tracker<10:
                    bb_time = back_burner_tracker*UnicornPy.SamplingRate
                    bb_new = bb_time+epoch_samples
                    back_burner[bb_time:bb_new,:] = curr_epoch
                    back_burner_tracker += 1
        
                else:
                    back_burner[0:-epoch_samples,:] = back_burner[epoch_samples:back_burner_samples,:]
                    back_burner[-epoch_samples:back_burner_samples,:] = curr_epoch
                    curr_cov = preprocess_raw(back_burner.T, epoch_samples)
                    curr_cov = curr_cov[None, :, :]
                    curr_pred = str(clf.predict(curr_cov)[0])
                    s.sendto(curr_pred.encode('utf-8'), server)
                    
            
                curr_epoch = np.zeros((epoch_samples, numberOfAcquiredChannels))
                curr_epoch_tracker = 0
                
            # np.savetxt(file,data,delimiter=',',fmt='%.3f',newline='\n')
            

        # Stop data acquisition.
        #-------------------------------------------------------------------------------------
        device.StopAcquisition();
        print()
        print("Data acquisition stopped.");

    except UnicornPy.DeviceException as e:
        print(e)
    except Exception as e:
        print("An unknown error occured. %s" %e)
    finally:
        # release receive allocated memory of receive buffer
        del receiveBuffer

        # Close device.
        #-------------------------------------------------------------------------------------
        del device
        print("Disconnected from Unicorn")

except UnicornPy.DeviceException as e:
    print(e)
except Exception as e:
    print("An unknown error occured. %s" %e)

print('Acquisition Over')



Trying to connect to 'UN-2019.05.78'.
Connected to 'UN-2019.05.78'.

Acquisition Configuration:
Sampling Rate: 250 Hz
Frame Length: 1
Number Of Acquired Channels: 17
Data Acquisition Length: 1 s

Data acquisition started.
Disconnected from Unicorn


KeyboardInterrupt: 